# Choreography Builders

Instead of directly editing protobufs, the spot_choreo_utils library provides pythonic builders that make it easy to programatically build and perform high level validations without a robot.

### Loading an existing animation

Animation builders accept existing animations, unlocking all of the utility functions that come with animation builders for existing animations

In [ ]:
import logging
from pathlib import Path
from spot_choreo_utils.choreo_creation.choreo_builders.animation_builder import AnimationBuilder
from spot_choreo_utils.paths import get_example_choreo_path
from spot_choreo_utils.serialization.serialization_utils import load_animation


#####
# Build from base animation
#####
ANIMATION_PATH = Path(get_example_choreo_path(), "animations", "pose_to_pose_animation.pbtxt")
existing_animation_proto = load_animation(ANIMATION_PATH)
logger = logging.Logger("animation_builder_logger")

existing_builder = AnimationBuilder(logger)
existing_builder.start_from_animation(existing_animation_proto)
animation_builder_proto = existing_builder.build()
print(animation_builder_proto)

Animation builders provide high level validation for the animation everytime it's built. You can toggle this feature, as well as other common animation settings by passing the BuildSetting class into the build_animation call. You can also call explicit validation checks prior to building the final animation. While it doesn't guarantee that the animation will play back successfully on robot, it does catch common issuses related to missing or ill defined properties.

In [ ]:
# You can explicitly check animation validity, but it is automatically checked in the build step
print(existing_builder.validate())
# Introduce an issue into the animation
existing_builder.raw_keyframes[0].time = 3
print(existing_builder.validate())

# Pass custom settings to the builder - turning off only output valid will
# allow the builder to return animations that don't pass offline validation
build_settings = AnimationBuilder.BuildSettings()
build_settings.only_output_valid = False
build_settings.apply_unique_name = False
build_settings.hold_final_pose_s = 3
animation_builder_proto = existing_builder.build(build_settings)
print(animation_builder_proto)

### Building from Scratch

Animation builders also massively reduce the amount of code required to create animations and sequences programatically.

In [ ]:
gripper_builder = AnimationBuilder(logger)
gripper_builder.start_from_empty(name="gripper_open_close")
gripper_builder.animate_gripper(start_time=0, gripper_angle=0.0)
gripper_builder.animate_gripper(start_time=1, gripper_angle=-1.0)
gripper_builder.animate_gripper(start_time=2, gripper_angle=0.0)

gripper_animation = gripper_builder.build()
print(gripper_animation)

The sequence builder handles building and validating the choreography sequence proto

In [ ]:
from spot_choreo_utils.choreo_creation.choreo_builders.sequence_builder import SequenceBuilder
import logging
import sys

logger = logging.Logger("Sequence builder")

sequence_builder = SequenceBuilder(logger)
sequence_builder.start_from_empty(name="gripper_open_close")
sequence_builder.add_animation(animation=gripper_animation, start_time=0)

gripper_sequence = sequence_builder.build()
print(gripper_sequence)